In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("0728test.csv")
df = df.drop(columns='Number')
REF_df = df.dropna()
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))
print(df.dtypes.value_counts())


has 875 null in all data


float64    218
object      44
int64        5
dtype: int64


In [2]:
cat = df.select_dtypes(include='object').columns
num = df.select_dtypes(include=['float64','int64']).columns

# Fill NA   (cat and num)

####  RNN

In [3]:
from sklearn.neighbors import RadiusNeighborsRegressor,KNeighborsRegressor
dis = 3
for i in range(df.shape[0]):
    Na_List = set(df.columns[df.iloc[i].isna() ].tolist())
    Na_List-=set(cat)
    for j in list(Na_List):
        i2 = df.columns.get_loc(j)
        if '_C_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-6:],REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(np.array(df.iloc[i,-6:]).reshape(1, -1))
        elif '_A1_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-6].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-6].reshape(-1, 1))
        elif '_A2_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-5].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-5].reshape(-1, 1))
        elif '_A3_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-4].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-4].reshape(-1, 1))
        elif '_A4_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-3].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-3].reshape(-1, 1))
        elif '_A5_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-2].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-2].reshape(-1, 1))
        elif '_A6_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-1].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-1].reshape(-1, 1))

In [4]:
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))

has 605 null in all data




# Fill category

In [5]:
def Convert(s):
    s=str(s)
    L=s.split(';')
    out=''
    if len(L)==1:
#         print(i)
        return 'EMPTY'
    if L[0]=='U':
        out+=str(L[1])
    elif L[0]=='D':
        out+='-'+str(L[1])
    else:
        out+='0'
    out+=';'
    if L[2]=='R':
        out+=str(L[3])
    elif L[2]=='L':
        out+='-'+str(L[3])
    else:
        out+='0'
    return out

In [6]:
# make REF
cat2=[]
for i in cat:
    REF_df[i]=REF_df[i].apply(Convert)
    REF_df['%s_x'%i]=REF_df['%s'%i].apply(lambda x: np.nan if pd.isna(x) else pd.Series(x.split(';')[0])).astype('float')
    REF_df['%s_y'%i]=REF_df['%s'%i].apply(lambda x: np.nan if pd.isna(x) else pd.Series(x.split(';')[1])).astype('float')
    cat2.append('%s_x'%i)
    cat2.append('%s_y'%i)
    REF_df=REF_df.drop(columns=[i])
REF_df = REF_df.reindex_axis(sorted(REF_df.columns), axis=1)

C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [7]:
cat2=[]
for i in cat:
    df[i]=df[i].apply(Convert)
    df['%s_x'%i]=df['%s'%i].apply(lambda x: REF_df['%s_x'%i].mean() if 'EMPTY' in x else (x.split(';')[0])).astype('float')
    df['%s_y'%i]=df['%s'%i].apply(lambda x: REF_df['%s_y'%i].mean() if 'EMPTY' in x else (x.split(';')[1])).astype('float')
    
    cat2.append('%s_x'%i)
    cat2.append('%s_y'%i)
    
    df=df.drop(columns=[i])
df = df.reindex_axis(sorted(df.columns), axis=1)

C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  # This is added back by InteractiveShellApp.init_path()


In [8]:
from sklearn.neighbors import RadiusNeighborsRegressor,KNeighborsRegressor
dis = 3
for i in range(df.shape[0]):
    Na_List = set(df.columns[df.iloc[i].isna() ].tolist())
    for j in list(Na_List):
        i2 = df.columns.get_loc(j)
        neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
        neigh.fit(REF_df.iloc[:,-6:],REF_df.iloc[:,i2])
        df.iloc[i,i2]=neigh.predict(np.array(df.iloc[i,-6:]).reshape(1, -1))

In [9]:
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))


has 0 null in all data




# Pred

In [10]:
import joblib
mapping = {'A1':[0],'A2':[1,2,3],'A3':[4,5,6,7,8],'A6':[9,10,11,12],'C':[13,14,15,16,17,18,19]}
result = pd.read_excel('109029_TestResult.xlsx')
result_original_order = result.columns
result.columns
result = result.reindex_axis(sorted(result.columns), axis=1)


C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  


In [11]:
reg_A1 = joblib.load('BEST_A1.sav')
reg_A2 = joblib.load('BEST_A2.sav')
reg_A3 = joblib.load('BEST_A3.sav')
reg_A6 = joblib.load('BEST_A6.sav')
reg_C = joblib.load('BEST_C.sav')

C:\Users\simon880502\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.gradient_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\simon880502\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\simon880502\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DummyRegressor from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\simon880502\Anaconda3\lib

In [12]:
result.iloc[:,mapping['A1']] = reg_A1.predict(df)
result.iloc[:,mapping['A2']] = reg_A2.predict(df)
result.iloc[:,mapping['A3']] = reg_A3.predict(df)
result.iloc[:,mapping['A6']] = reg_A6.predict(df)
result.iloc[:,mapping['C']] = reg_C.predict(df)

In [13]:
result = result.reindex_axis(result_original_order, axis=1)
result = np.round(result,5)
result.to_excel('109029_TestResult.xlsx',index=False)

C:\Users\simon880502\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.
